In [1]:
import openai
import json
import yaml

In [2]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/openai-keys.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Applying our API key and organization ID to OpenAI
openai.organization = keys_yaml['ORG_ID']
openai.api_key = keys_yaml['API_KEY']

In [3]:
with open('../data/about-me.txt', 'r') as f:
    about_me = f.read()

about_me

'Hello! My name is David Hundley. I am a principal machine learning engineer at State Farm. I enjoy learning about AI and teaching what I learn back to others. I have two daughters. I drive a Tesla Model 3, and my favorite video game series is The Legend of Zelda.'

In [4]:
about_me_prompt = f'''
Please extract information as a JSON object. Please look for the following pieces of information:
Name
Job title
Company
Number of children as a single integer
Car make
Car model
Favorite video game series

This is the body of text to extract the information from:
{about_me}
'''

In [5]:
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me_prompt}]
)

openai_response

<OpenAIObject chat.completion id=chatcmpl-7a3uMnJrdc5gmErAGynuRreOiQQRO at 0x108420f90> JSON: {
  "id": "chatcmpl-7a3uMnJrdc5gmErAGynuRreOiQQRO",
  "object": "chat.completion",
  "created": 1688829186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"Name\": \"David Hundley\",\n  \"Job title\": \"Principal Machine Learning Engineer\",\n  \"Company\": \"State Farm\",\n  \"Number of children\": 2,\n  \"Car make\": \"Tesla\",\n  \"Car model\": \"Model 3\",\n  \"Favorite video game series\": \"The Legend of Zelda\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 122,
    "completion_tokens": 70,
    "total_tokens": 192
  }
}

In [6]:
json.loads(openai_response['choices'][0]['message']['content'])

{'Name': 'David Hundley',
 'Job title': 'Principal Machine Learning Engineer',
 'Company': 'State Farm',
 'Number of children': 2,
 'Car make': 'Tesla',
 'Car model': 'Model 3',
 'Favorite video game series': 'The Legend of Zelda'}

In [10]:
string1 = 'David'
string2 = 'david'
string3 = 'David.'
string4 = 'David'

In [11]:
string1 == string2

False

In [12]:
string1 == string4

True

In [39]:
functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text.',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of person' 
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    },
    {
        'name': 'extract_car_info',
        'description': 'Extract the make and model of the person\'s car',
        'parameters': {
            'type': 'object',
            'properties': {
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                }
            }
        }
    },
    {
        'name': 'extract_all_info',
        'description': 'Extract all information about a person including their vehicle make and model',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                },
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                },
                'company_name': {
                    'type': 'string',
                    'description': 'Name of the company the person works for'
                },
                'favorite_vg_series': {
                    'type': 'string',
                    'description': 'Name of the person\'s favorite video game series'
                }
            }
        }
    }
]

In [21]:
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me}],
    functions = functions,
    function_call = 'auto'
)

openai_response

<OpenAIObject chat.completion id=chatcmpl-7a4CdkHofcT03WPSOVmRRTj8jcEYp at 0x11a57d3a0> JSON: {
  "id": "chatcmpl-7a4CdkHofcT03WPSOVmRRTj8jcEYp",
  "object": "chat.completion",
  "created": 1688830319,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extract_person_info",
          "arguments": "{\n  \"name\": \"David Hundley\",\n  \"job_title\": \"Principal Machine Learning Engineer\",\n  \"num_children\": 2\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 146,
    "completion_tokens": 37,
    "total_tokens": 183
  }
}

In [23]:
json_response = json.loads(openai_response['choices'][0]['message']['function_call']['arguments'])

In [25]:
json_response

{'name': 'David Hundley',
 'job_title': 'Principal Machine Learning Engineer',
 'num_children': 2}

In [26]:
def extract_person_info(name, job_title, num_children):
    return f'The person\'s name is {name}. Their job title is {job_title}. They have {num_children} children.'

In [33]:
openai_response

<OpenAIObject chat.completion id=chatcmpl-7a4CdkHofcT03WPSOVmRRTj8jcEYp at 0x11a57d3a0> JSON: {
  "id": "chatcmpl-7a4CdkHofcT03WPSOVmRRTj8jcEYp",
  "object": "chat.completion",
  "created": 1688830319,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extract_person_info",
          "arguments": "{\n  \"name\": \"David Hundley\",\n  \"job_title\": \"Principal Machine Learning Engineer\",\n  \"num_children\": 2\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 146,
    "completion_tokens": 37,
    "total_tokens": 183
  }
}

In [37]:
if openai_response["choices"][0]["message"].get('function_call'):

    json_response = json.loads(openai_response['choices'][0]['message']['function_call']['arguments'])

    available_functions = {
        'extract_person_info': extract_person_info,
        'extract_car_information'
    }

    function_to_call = openai_response['choices'][0]['message']['function_call']['name']

    print(available_functions[function_to_call](json_response['name'], json_response['job_title'], json_response['num_children']))

The person's name is David Hundley. Their job title is Principal Machine Learning Engineer. They have 2 children.


In [43]:
openai_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': 'My name is David Hundley. I work as a principal machine learning engineer. I have two daughters.'}],
    functions = functions,
    function_call = 'auto'
)

openai_response

<OpenAIObject chat.completion id=chatcmpl-7a4SSkw8TErZcpeblPvzPDBRd1ps7 at 0x10a718720> JSON: {
  "id": "chatcmpl-7a4SSkw8TErZcpeblPvzPDBRd1ps7",
  "object": "chat.completion",
  "created": 1688831300,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extract_person_info",
          "arguments": "{\n  \"name\": \"David Hundley\",\n  \"job_title\": \"Principal Machine Learning Engineer\",\n  \"num_children\": 2\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 281,
    "completion_tokens": 37,
    "total_tokens": 318
  }
}